# Airline delays 
## Bureau of Transportation Statistics
https://www.transtats.bts.gov/OT_Delay/OT_DelayCause1.asp   
https://www.bts.gov/topics/airlines-and-airports/understanding-reporting-causes-flight-delays-and-cancellations

~140GB

In [2]:
# imports
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [3]:
#dbutils.fs.ls('/databricks-datasets/airlines')

In [4]:
sum = 0
DATA_PATH = 'dbfs:/databricks-datasets/airlines/'
for item in dbutils.fs.ls(DATA_PATH):
  sum = sum+item.size
sum

Out[3]: 128911081815

In [5]:
# with open("/dbfs/databricks-datasets/airlines/README.md") as f:
#     x = ''.join(f.readlines())

# print(x)

In [6]:
from pyspark.sql.types import *
airline_schema = StructType([
            StructField("Year", IntegerType(), True),
            StructField("Month", IntegerType(), True),
            StructField("DayofMonth", IntegerType(), True),
            StructField("DayOfWeek", IntegerType(), True),
            StructField("DepTime", IntegerType(), True),
            StructField("CRSDepTime", IntegerType(), True),
            StructField("ArrTime", IntegerType(), True),
            StructField("CRSArrTime", IntegerType(), True),
            StructField("UniqueCarrier", StringType(), True),
            StructField("FlightNum", StringType(), True),
            StructField("TailNum", StringType(), True),
            StructField("ActualElapsedTime", IntegerType(), True),
            StructField("CRSElapsedTime", IntegerType(), True),
            StructField("AirTime", IntegerType(), True),
            StructField("ArrDelay", IntegerType(), True),
            StructField("DepDelay", IntegerType(), True),
            StructField("Origin", StringType(), True),
            StructField("Dest", StringType(), True),
            StructField("Distance", IntegerType(), True),
            StructField("TaxiIn", IntegerType(), True),			
			StructField("TaxiOut", IntegerType(), True),
            StructField("Cancelled", IntegerType(), True),
            StructField("CancellationCode", StringType(), True),
            StructField("Diverted", IntegerType(), True),
            StructField("CarrierDelay", IntegerType(), True),
            StructField("WeatherDelay", IntegerType(), True),
            StructField("NASDelay", IntegerType(), True),
            StructField("SecurityDelay", IntegerType(), True),
			StructField("LateAircraftDelay", IntegerType(), True),
            StructField("IsArrDelayed", StringType(), True),			
            StructField("IsDepDelayed", StringType(), True)])

In [7]:
airlines = spark.read.format("csv").option("header", "false").schema(airline_schema).load("dbfs:/databricks-datasets/airlines/part-*")
airlines.printSchema()

root
-- Year: integer (nullable = true)
-- Month: integer (nullable = true)
-- DayofMonth: integer (nullable = true)
-- DayOfWeek: integer (nullable = true)
-- DepTime: integer (nullable = true)
-- CRSDepTime: integer (nullable = true)
-- ArrTime: integer (nullable = true)
-- CRSArrTime: integer (nullable = true)
-- UniqueCarrier: string (nullable = true)
-- FlightNum: string (nullable = true)
-- TailNum: string (nullable = true)
-- ActualElapsedTime: integer (nullable = true)
-- CRSElapsedTime: integer (nullable = true)
-- AirTime: integer (nullable = true)
-- ArrDelay: integer (nullable = true)
-- DepDelay: integer (nullable = true)
-- Origin: string (nullable = true)
-- Dest: string (nullable = true)
-- Distance: integer (nullable = true)
-- TaxiIn: integer (nullable = true)
-- TaxiOut: integer (nullable = true)
-- Cancelled: integer (nullable = true)
-- CancellationCode: string (nullable = true)
-- Diverted: integer (nullable = true)
-- CarrierDelay: integer (nullable = true)
-- WeatherDelay: integer (nullable = true)
-- NASDelay: integer (nullable = true)
-- SecurityDelay: integer (nullable = true)
-- LateAircraftDelay: integer (nullable = true)
-- IsArrDelayed: string (nullable = true)
-- IsDepDelayed: string (nullable = true)

In [8]:
start_time = time.time()
count =  airlines.count()
total_time_csv = time.time() - start_time
print("Total records: {} \n Time taken (in seconds) to count total records: {}".format(count, np.round(total_time_csv, 2) )) 

Total records: 1235349691 
 Time taken (in seconds) to count total records: 106.59

In [9]:
airlines.write.parquet("dbfs:/tmp/parquet/airlines_data_in_parquet.parquet")

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o2440.parquet.
: org.apache.spark.sql.AnalysisException: path dbfs:/tmp/parquet/airlines_data_in_parquet.parquet already exists.;
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:148)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:126)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:146)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:134)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$5.apply(SparkPlan.scala:187)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:183)
	at org.apache.spark.sql.execution.SparkPlan.execute(SparkPlan.scala:134)
	at org.apache.spark.sql.execution.QueryExecution.toRdd$lzycompute(QueryExecution.scala:115)
	at org.apache.spark.sql.execution.QueryExecution.toRdd(QueryExecution.scala:115)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(DataFrameWriter.scala:710)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(DataFrameWriter.scala:710)
	at org.apache.spark.sql.execution.SQLExecution$$anonfun$withCustomExecutionEnv$1.apply(SQLExecution.scala:111)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:240)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:97)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:170)
	at org.apache.spark.sql.DataFrameWriter.runCommand(DataFrameWriter.scala:710)
	at org.apache.spark.sql.DataFrameWriter.saveToV1Source(DataFrameWriter.scala:306)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:292)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:235)
	at org.apache.spark.sql.DataFrameWriter.parquet(DataFrameWriter.scala:600)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 AnalysisException Traceback (most recent call last)
 <command-2392600259027127> in <module> 
 ----> 1 airlines . write . parquet ( "dbfs:/tmp/parquet/airlines_data_in_parquet.parquet" ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in parquet (self, path, mode, partitionBy, compression) 
 842 self . partitionBy ( partitionBy ) 
 843 self . _set_opts ( compression = compression ) 
 --> 844 self . _jwr

In [10]:
airlines_df = spark.read.parquet("dbfs:/tmp/parquet/airlines_data_in_parquet.parquet")


In [11]:
airlines_df.show(5)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|IsArrDelayed|IsDepDelayed|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
2006| 12| 12| 2| 650| 610| 807| 735| YV| 7315| N154SF| 77| 85| 47| 32| 40| JFK| IAD| 228| 9| 21| 0| null| 0| 32| 0| 0| 0| 0| YES| YES|
2006| 12| 12| 2| 1856| 1853| 2013| 2023| YV| 7318| N37342| 77| 90| 47| -10| 3| JFK| IAD| 228| 7| 23| 0| null| 0| 0| 0| 0| 0| 0| NO| YES|
2006| 12| 12| 2| 1215| 1225| 1340| 1345| YV| 7346| N47202| 85| 80| 53| -5| -10| JFK| IAD| 228| 8| 24| 0| null| 0| 0| 0| 0| 0| 0| NO| NO|
2006| 12| 12| 2| 2000| 1655| 2120| 1824| YV| 7381| N47202| 80| 89| 44| 176| 185| JFK| IAD| 228| 7| 29| 0| null| 0| 0| 0| 0| 0| 176| YES| YES|
2006| 12| 12| 2| 813| 820| 947| 945| YV| 7417| N75994| 94| 85| 44| 2| -7| JFK| IAD| 228| 5| 45| 0| null| 0| 0| 0| 0| 0| 0| YES| NO|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
only showing top 5 rows

In [12]:
airlines_df.printSchema()

root
-- Year: integer (nullable = true)
-- Month: integer (nullable = true)
-- DayofMonth: integer (nullable = true)
-- DayOfWeek: integer (nullable = true)
-- DepTime: integer (nullable = true)
-- CRSDepTime: integer (nullable = true)
-- ArrTime: integer (nullable = true)
-- CRSArrTime: integer (nullable = true)
-- UniqueCarrier: string (nullable = true)
-- FlightNum: string (nullable = true)
-- TailNum: string (nullable = true)
-- ActualElapsedTime: integer (nullable = true)
-- CRSElapsedTime: integer (nullable = true)
-- AirTime: integer (nullable = true)
-- ArrDelay: integer (nullable = true)
-- DepDelay: integer (nullable = true)
-- Origin: string (nullable = true)
-- Dest: string (nullable = true)
-- Distance: integer (nullable = true)
-- TaxiIn: integer (nullable = true)
-- TaxiOut: integer (nullable = true)
-- Cancelled: integer (nullable = true)
-- CancellationCode: string (nullable = true)
-- Diverted: integer (nullable = true)
-- CarrierDelay: integer (nullable = true)
-- WeatherDelay: integer (nullable = true)
-- NASDelay: integer (nullable = true)
-- SecurityDelay: integer (nullable = true)
-- LateAircraftDelay: integer (nullable = true)
-- IsArrDelayed: string (nullable = true)
-- IsDepDelayed: string (nullable = true)

In [13]:
start_time_parquet = time.time()
count = airlines_df.count()
total_time_parquet = time.time() - start_time_parquet
print("Total records: {} \n Time taken (in seconds) to count total records: {} \n Parquet read DataFrame is {} times faster than CSV read DataFrame".format(count, np.round(total_time_parquet, 2), np.round(total_time_csv/total_time_parquet, 2) )) 

Total records: 1235349691 
 Time taken (in seconds) to count total records: 0.94 
 Parquet read DataFrame is 112.88 times faster than CSV read DataFrame

In [14]:
def nullDataFrame(df):
  null_feature_list = []
  for column in df.columns:
    nulls = df.filter(df[column].isNull()).count()
    nulls_perct = np.round((nulls/count)*100, 2)
    null_feature_list.append([column, nulls, nulls_perct])
  nullCounts_df = pd.DataFrame(np.array(null_feature_list), columns=['Feature_Name', 'Null_Counts', 'Percentage_Null_Counts'])
  return nullCounts_df  

In [15]:
nullCounts_df = nullDataFrame(airlines_df)

In [16]:
nullCounts_df

,Feature_Name,Null_Counts,Percentage_Null_Counts
0,Year,899947541,72.85
1,Month,899947541,72.85
2,DayofMonth,899947541,72.85
3,DayOfWeek,899947541,72.85
4,DepTime,899947541,72.85
5,CRSDepTime,899947541,72.85
6,ArrTime,899947541,72.85
7,CRSArrTime,899947541,72.85
8,UniqueCarrier,899947541,72.85
9,FlightNum,899947541,72.85


In [17]:
subset = [x for x in airlines_df.columns if x not in ["CancellationCode"]]
airlines_df2 = airlines_df.na.drop(subset=subset)

In [18]:
airlines_df2.count()

Out[17]: 335402130

In [19]:
nullCounts_df2 = nullDataFrame(airlines_df2)
nullCounts_df2

,Feature_Name,Null_Counts,Percentage_Null_Counts
0,Year,0,0.0
1,Month,0,0.0
2,DayofMonth,0,0.0
3,DayOfWeek,0,0.0
4,DepTime,0,0.0
5,CRSDepTime,0,0.0
6,ArrTime,0,0.0
7,CRSArrTime,0,0.0
8,UniqueCarrier,0,0.0
9,FlightNum,0,0.0


In [20]:
Delay_List = ['ArrDelay', 'DepDelay', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay' ]
airlines_df2[Delay_List].describe().show()

+-------+-----------------+------------------+------------------+------------------+------------------+--------------------+
summary| ArrDelay| DepDelay| CarrierDelay| WeatherDelay| NASDelay| SecurityDelay|
+-------+-----------------+------------------+------------------+------------------+------------------+--------------------+
 count| 335402130| 335402130| 335402130| 335402130| 335402130| 335402130|
 mean|9.962511657275403|10.859205426035905| 3.7553481845807|0.8039703862345776| 4.184947334711321|0.027236559290783276|
 stddev|38.10062801544172| 34.83892757575155|20.047482279328964| 9.526681798236828|16.748926861184817| 1.205579251023994|
 min| -1302| -1200| 0| 0| -60| 0|
 max| 2598| 2601| 2580| 1510| 1392| 533|
+-------+-----------------+------------------+------------------+------------------+------------------+--------------------+

In [21]:
sample_airlines_df = airlines_df2.select(Delay_List).sample(False, 0.1, 2020)
pandas_df = sample_airlines_df.toPandas()

In [22]:
pandas_df.hist(figsize=(15,15), bins=50)
display(plt.show())